In [7]:
from torch.autograd import Variable
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import cv2 
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
import pandas as pd
import seaborn as sn

In [8]:

def loadDataset(path,Train_size,Val_size,Test_size,batch_,Shuffle):

    
    _transforms = transforms.Compose([transforms.Grayscale(num_output_channels=1),transforms.Resize((img_Resize,img_Resize)),transforms.ToTensor(),
                                       transforms.Normalize(0.1307,0.3081)])
     
    data_train = datasets.ImageFolder((data_dir + '/' + 'train') , transform=_transforms)
    data_test=datasets.ImageFolder((data_dir+'/'+'test'), transform=_transforms)
    
    train_data, val_data = torch.utils.data.random_split(data_train, [Train_size, Val_size])

    test_size=list(range(0, Test_size))
    

    test_data=torch.utils.data.Subset(data_test,test_size)
    
    print(len(train_data))
    print(len(val_data))
    print(len(test_data))
    
    
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_, shuffle=True)
    val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_,shuffle=True)
    test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_,shuffle=True)
        
    return train_loader,val_loader,test_loader

In [9]:

def init_network(no_of_layers,input_dim,neurons_per_layer,droupout):
    a=input_dim
    b=neurons_per_layer[-1]
    c=neurons_per_layer
    d=droupout
    
    return a,b,c,d

A,B,C,D=init_network(2, 784, [100, 50,10], 0.1)

class Net(nn.Module):
    
    def __init__(self, input_dim=A, output_dim=B, hidden_dim=C,droup_out=D):
        super(Net, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        current_dim = input_dim
        self.layers = nn.ModuleList()
        for hdim in hidden_dim:
            if(hdim==hidden_dim[-1]):
                self.layers.append(nn.Dropout(droup_out))
                self.layers.append(nn.Linear(current_dim, hdim))
                current_dim = hdim
                
            else:
                self.layers.append(nn.Linear(current_dim, hdim))
                current_dim = hdim


    def forward(self, x):
        for layer in self.layers[:-1]:
            x = F.leaky_relu(layer(x))
        out = F.softmax(self.layers[-1](x))
        return out   
     

In [10]:
def save_network(model):
    torch.save(model.state_dict(), "weights.pth")
    return

In [11]:

def train(net, train_loader, val_loader ,training_epochs, loss_func, optimizer):
    
    print(net)
    train_loss_array=[]
    train_loss_epoch=[]
    train_accuracy_epoch=[]
    val_loss_array=[]
    val_loss_epoch=[]
    val_accuracy_epoch=[]
    criterion=loss_func 
    
    for epoch in range(training_epochs):
        net.train()
        correct = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = Variable(data), Variable(target)
            data, target = data.to(device), target.to(device)        
            data = data.view(-1, img_Resize*img_Resize*1)
            optimizer.zero_grad()
            net_out = net(data)
            pred = net_out.data.max(1)[1] 
            loss = criterion(net_out, target)
            correct += pred.eq(target.data).sum()
            loss.backward()
            optimizer.step()
            if batch_idx % 10 == 0:
                print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        epoch, batch_idx * len(data), len(train_loader.dataset),
                               100. * batch_idx / len(train_loader), loss.item()))
                train_loss_array.append(loss.item())
            
        train_accuracy_=float(100 * correct / len(train_loader.dataset))
        train_accuracy_epoch.append(train_accuracy_)
        epoch_loss=np.mean(train_loss_array)
        train_loss_array=[]
        train_loss_epoch.append(epoch_loss)
        print(train_loss_epoch)
        print(train_accuracy_epoch)

        correct=0
        criterion=loss_func

        net.eval()
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(val_loader):
                data, target = Variable(data), Variable(target)
                data, target = data.to(device), target.to(device)        
                data = data.view(-1, img_Resize*img_Resize*1)
                net_out = net(data)
                pred = net_out.data.max(1)[1]
    
                loss = criterion(net_out, target)
                correct += pred.eq(target.data).sum()
                if batch_idx % 10 == 0:
                    print('Val Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                            epoch, batch_idx * len(data), len(val_loader.dataset),
                                   100. * batch_idx / len(val_loader), loss.item()))
                    val_loss_array.append(loss.item())
                
            val_accuracy_=float(100 * correct / len(val_loader.dataset))
            val_accuracy_epoch.append(val_accuracy_)
            epoch_loss=np.mean(val_loss_array)
            val_loss_array=[]
            val_loss_epoch.append(epoch_loss)
            print(val_loss_epoch)
            print(val_accuracy_epoch)
            
            save_network(net)

    return net, train_loss_epoch, train_accuracy_epoch, val_loss_epoch, val_accuracy_epoch


In [12]:
    def load_model(net,path):   
        model = Net()
        model.load_state_dict(torch.load(path))
        return model

In [13]:

def test(model,test_set):
    tar=[]
    test_loss = 0
    correct = 0
    criterion=nn.CrossEntropyLoss() 
    predicted=[]
    

    with torch.no_grad():
        for data, target in test_loader:
            data, target = Variable(data), Variable(target)
            data, target = data.to(device), target.to(device)
    
            data = data.view(-1, img_Resize * img_Resize*1)
            net_out = model(data)
            # sum up batch loss
            test_loss += criterion(net_out, target).item()
            pred = net_out.data.max(1)[1]  # get the index of the max log-probability
            p=pred.tolist()
            t=target.tolist()
            predicted=predicted+p
            tar=tar+t
            correct += pred.eq(target.data).sum()
    
        test_loss /= len(test_loader.dataset)
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
                test_loss, correct, len(test_loader.dataset),
                100. * correct / len(test_loader.dataset)))
        
        examples = enumerate(test_loader)
        batch_idx, (example_data, example_targets) = next(examples)
        
        fig1 = plt.figure()
        for i in range(6):
          plt.subplot(2,3,i+1)
          plt.tight_layout()
          plt.imshow(example_data[i][0], cmap='gray', interpolation='none')
          plt.title("Ground Truth: {}".format(example_targets[i]))
          plt.xticks([])
          plt.yticks([])
        fig1

    return predicted, tar



In [14]:

def visualize(pred,target,train_loss=None,train_Acc=None,Val_loss=None,Val_Acc=None):
    

    f1=f1_score(target, pred,average=None)
    f1=sum(f1)/len(f1)
    print(f1)
    cm=confusion_matrix(target, pred)
    df_cm = pd.DataFrame(cm, index = [i for i in "0123456789"],
                  columns = [i for i in "0123456789"])
    plt.figure(figsize = (10,7))
    sn.heatmap(df_cm, annot=True, linewidths=.5,fmt="d")
    

    plt.title('Confusion Matrix', fontsize = 20) # title with fontsize 20
    plt.xlabel('Predicted Labels', fontsize = 15) # x-axis label with fontsize 15
    plt.ylabel('True Labels', fontsize = 15) # y-axis label with fontsize 15
    plt.show()
    

    
    if train_loss != None:
        x1= Val_Acc
        x2= train_Acc
        x=[1,2,3,4,5,6,7,8,9,10]
        plt.plot(x, x1, label = "val_acc")
        plt.plot(x, x2, label = "train_acc" )
        plt.title('Accuracy Comparison', fontsize=15)
        plt.xlabel('Number of Epochs', fontsize=15)
        plt.ylabel('Accuracy %', fontsize=15)
        plt.legend(loc="upper left")
        plt.show() 

        # plot lines 
        x1= Val_loss
        x2= train_loss
        x=[1,2,3,4,5,6,7,8,9,10]
        # plot lines
        plt.plot(x, x1, label = "val_loss")
        plt.plot(x, x2, label = "train_loss" )
        plt.title('Loss Comparison', fontsize=15)
        plt.xlabel('Number of Epochs', fontsize=15)
        plt.ylabel('Loss', fontsize=15)
        plt.legend(loc="upper right")
        plt.show() 
    return


In [15]:

def main(data_dir,size_train,size_val,size_test,Batch_Size,hidden,neural_list,isGPU,dropout,
         is_Train,Visualizer,trainEpochs,Loss_func,
         optimizer_,learning_rate_):
    
    img_Resize=28
    t_loader,v_loader,test_loader=loadDataset(data_dir, size_train, size_val, size_test, Batch_Size,Shuffle=True)
    A,B,C,D=init_network(hidden,img_Resize*img_Resize,neural_list,dropout )
    class Net(nn.Module):
    
        def __init__(self, input_dim=A, output_dim=B, hidden_dim=C,droup_out=D):
            super(Net, self).__init__()
            self.input_dim = input_dim
            self.output_dim = output_dim
            self.hidden_dim = hidden_dim
            current_dim = input_dim
            self.layers = nn.ModuleList()
            for hdim in hidden_dim:
                if(hdim==hidden_dim[-1]):
                    self.layers.append(nn.Dropout(droup_out))
                    self.layers.append(nn.Linear(current_dim, hdim))
                    current_dim = hdim

                else:
                    self.layers.append(nn.Linear(current_dim, hdim))
                    current_dim = hdim


        def forward(self, x):
            for layer in self.layers[:-1]:
                x = F.leaky_relu(layer(x))
            out = F.softmax(self.layers[-1](x))
            
            return out 
    
    model = Net()
    print(model)
    learning_rate=0.05
    epochs=10
    
    if isGPU==True:
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        model.to(device)
    
    if is_Train==True:
        model,train_loss,train_acc,val_loss,val_acc = train(model, t_loader,v_loader, trainEpochs, nn.CrossEntropyLoss(), optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9))
    else:
        # this loads the saved model from give directory.
        model = Net()
        train_acc=[]
        model.load_state_dict(torch.load('C:/Users/Abdullah/.spyder-py3/weights.pth'))
        
    pred,target=test(model,test_loader)
    if Visualizer==True and len(train_acc)>0:
        visualize(pred,target,train_loss,train_acc,val_loss,val_acc)
    elif Visualizer==True:
        visualize(pred,target)
        
        
    return

In [16]:
data_dir='C:/Users/Abdullah/.spyder-py3/MNIST_Data'
print('Abdullah Aziz','MSDS20052')

Loss_func=nn.CrossEntropyLoss()
learning_rate_=0.5
optimizer_=optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
isGPU=False
dropout=0.1
is_Train=True
Visualizer=True
trainEpochs=10


main(data_dir,size_train,size_val,size_test,Batch_Size,hidden,neural_list,isGPU,dropout,is_Train,Visualizer,trainEpochs,
     Loss_func,optimizer_,learning_rate_)


Abdullah Aziz MSDS20052


NameError: name 'model' is not defined

In [21]:
data_dir='C:/Users/Abdullah/.spyder-py3/MNIST_Data'
print('Abdullah Aziz','MSDS20052')
Train_size=50000
Val_size=10000
Test_size=10000
batch_=64
hidden=2
neural_list=[100, 50,10]
img_Resize=28


_transforms = transforms.Compose([transforms.Grayscale(num_output_channels=1),transforms.Resize((img_Resize,img_Resize)),transforms.ToTensor(),
                                       transforms.Normalize(0.1307,0.3081)])

data_train = datasets.ImageFolder((data_dir + '/' + 'train') , transform=_transforms)
data_test=datasets.ImageFolder((data_dir+'/'+'test'), transform=_transforms)

train_data, val_data = torch.utils.data.random_split(data_train, [Train_size, Val_size])

test_size=list(range(0, Test_size))


test_data=torch.utils.data.Subset(data_test,test_size)

print(len(train_data))
print(len(val_data))
print(len(test_data))


train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=batch_,shuffle=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=batch_,shuffle=True)



Abdullah Aziz MSDS20052
50000
10000
10000


In [27]:
D=0.1
C=[100, 50,10]

class Net(nn.Module):
    
    def __init__(self, input_dim=784, output_dim=C[-1], hidden_dim=C,droup_out=D):
        super(Net, self).__init__()
        self.input_dim = input_dim
        self.output_dim = output_dim
        self.hidden_dim = hidden_dim
        current_dim = input_dim
        self.layers = nn.ModuleList()
        for hdim in hidden_dim:
            if(hdim==hidden_dim[-1]):
                self.layers.append(nn.Dropout(droup_out))
                self.layers.append(nn.Linear(current_dim, hdim))
                current_dim = hdim
                
            else:
                self.layers.append(nn.Linear(current_dim, hdim))
                current_dim = hdim


    def forward(self, x):
        for layer in self.layers[:-1]:
            x = F.leaky_relu(layer(x))
        out = F.softmax(self.layers[-1](x))
        return out   
     

In [28]:
net=Net()

In [37]:
train_loss_array=[]
train_loss_epoch=[]
train_accuracy_epoch=[]
val_loss_array=[]
val_loss_epoch=[]
val_accuracy_epoch=[]
training_epochs=2
criterion=nn.CrossEntropyLoss() 
device='cpu'
optimizer=optim.SGD(net.parameters(),lr=0.001, momentum=0.9)

for epoch in range(training_epochs):
    net.train()
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        data, target = data.to(device), target.to(device)        
        data = data.view(-1, img_Resize*img_Resize*1)
        optimizer.zero_grad()
        net_out = net(data)
        pred = net_out.data.max(1)[1] 
        loss = criterion(net_out, target)
        correct += pred.eq(target.data).sum()
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                    epoch, batch_idx * len(data), len(train_loader.dataset),
                           100. * batch_idx / len(train_loader), loss.item()))
            train_loss_array.append(loss.item())

    train_accuracy_=float(100 * correct / len(train_loader.dataset))
    train_accuracy_epoch.append(train_accuracy_)
    epoch_loss=np.mean(train_loss_array)
    train_loss_array=[]
    train_loss_epoch.append(epoch_loss)
    print(train_loss_epoch)
    print(train_accuracy_epoch)

    correct=0
    criterion=nn.CrossEntropyLoss()

    net.eval()
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(val_loader):
            data, target = Variable(data), Variable(target)
            data, target = data.to(device), target.to(device)        
            data = data.view(-1, img_Resize*img_Resize*1)
            net_out = net(data)
            pred = net_out.data.max(1)[1]

            loss = criterion(net_out, target)
            correct += pred.eq(target.data).sum()
            if batch_idx % 10 == 0:
                print('Val Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                        epoch, batch_idx * len(data), len(val_loader.dataset),
                               100. * batch_idx / len(val_loader), loss.item()))
                val_loss_array.append(loss.item())

        val_accuracy_=float(100 * correct / len(val_loader.dataset))
        val_accuracy_epoch.append(val_accuracy_)
        epoch_loss=np.mean(val_loss_array)
        val_loss_array=[]
        val_loss_epoch.append(epoch_loss)
        print(val_loss_epoch)
        print(val_accuracy_epoch)


<ipython-input-27-8b079b529ddd>:27: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  out = F.softmax(self.layers[-1](x))


Train Epoch: 0 [0/50000 (0%)]	Loss: 2.302569
Train Epoch: 0 [640/50000 (1%)]	Loss: 2.304336
Train Epoch: 0 [1280/50000 (3%)]	Loss: 2.302072
Train Epoch: 0 [1920/50000 (4%)]	Loss: 2.303528
Train Epoch: 0 [2560/50000 (5%)]	Loss: 2.300338
Train Epoch: 0 [3200/50000 (6%)]	Loss: 2.304111
Train Epoch: 0 [3840/50000 (8%)]	Loss: 2.300156
Train Epoch: 0 [4480/50000 (9%)]	Loss: 2.304207
Train Epoch: 0 [5120/50000 (10%)]	Loss: 2.300815
Train Epoch: 0 [5760/50000 (12%)]	Loss: 2.298398
Train Epoch: 0 [6400/50000 (13%)]	Loss: 2.300295
Train Epoch: 0 [7040/50000 (14%)]	Loss: 2.300983
Train Epoch: 0 [7680/50000 (15%)]	Loss: 2.298836
Train Epoch: 0 [8320/50000 (17%)]	Loss: 2.301413
Train Epoch: 0 [8960/50000 (18%)]	Loss: 2.300502
Train Epoch: 0 [9600/50000 (19%)]	Loss: 2.302307
Train Epoch: 0 [10240/50000 (20%)]	Loss: 2.299599
Train Epoch: 0 [10880/50000 (22%)]	Loss: 2.301039
Train Epoch: 0 [11520/50000 (23%)]	Loss: 2.299074
Train Epoch: 0 [12160/50000 (24%)]	Loss: 2.296423
Train Epoch: 0 [12800/50000 

Train Epoch: 1 [44800/50000 (90%)]	Loss: 1.968367
Train Epoch: 1 [45440/50000 (91%)]	Loss: 1.972341
Train Epoch: 1 [46080/50000 (92%)]	Loss: 2.003456
Train Epoch: 1 [46720/50000 (93%)]	Loss: 1.903667
Train Epoch: 1 [47360/50000 (95%)]	Loss: 1.880410
Train Epoch: 1 [48000/50000 (96%)]	Loss: 2.012703
Train Epoch: 1 [48640/50000 (97%)]	Loss: 2.008346
Train Epoch: 1 [49280/50000 (98%)]	Loss: 1.930649
Train Epoch: 1 [49920/50000 (100%)]	Loss: 1.893036
[2.2848923447765883, 2.096320918843716]
[25.577999114990234, 47.89400100708008]
Val Epoch: 1 [0/10000 (0%)]	Loss: 1.972398
Val Epoch: 1 [640/10000 (6%)]	Loss: 1.934266
Val Epoch: 1 [1280/10000 (13%)]	Loss: 1.914585
Val Epoch: 1 [1920/10000 (19%)]	Loss: 1.844798
Val Epoch: 1 [2560/10000 (25%)]	Loss: 1.844449
Val Epoch: 1 [3200/10000 (32%)]	Loss: 1.884656
Val Epoch: 1 [3840/10000 (38%)]	Loss: 1.984246
Val Epoch: 1 [4480/10000 (45%)]	Loss: 1.878670
Val Epoch: 1 [5120/10000 (51%)]	Loss: 1.856087
Val Epoch: 1 [5760/10000 (57%)]	Loss: 1.909438
Val E

In [ ]:
torch.save(net.state_dict(), "weights.pth")


In [38]:
cd

C:\Users\Abdullah
